In [ ]:
# 🔗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# ⬇️ Clonar AUTOMATIC1111 WebUI
!rm -rf /content/stable-diffusion-webui
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

In [ ]:
# 🩹 Sobrescrever requirements.txt (removendo tokenizers)
reqs = '''
torch
torchvision
torchaudio
transformers==4.40.1
accelerate
safetensors
''' 
with open('/content/stable-diffusion-webui/requirements.txt', 'w') as f:
    f.write(reqs)

In [ ]:
# 🔗 Conectar pasta de LoRAs
!mkdir -p /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/SD_LoRAs /content/stable-diffusion-webui/models/Lora

In [ ]:
# ▶️ Rodar em modo CPU (sem checagem CUDA)
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --precision full --no-half" \
  REQS_FILE="requirements.txt" \
  python launch.py